In [4]:
import cv2
import numpy as np

In [5]:
def find_intersection(p1, p2, p3, p4):
    a1 = p2[1] - p1[1]
    b1 = p1[0] - p2[0]
    c1 = a1*p1[0] + b1*p1[1]
    
    a2 = p4[1] - p3[1]
    b2 = p3[0] - p4[0]
    c2 = a2*p3[0] + b2*p3[1]
    
    determinant = a1*b2 - a2*b1
    # Determinant not zero
    if determinant:
        x = round((b2*c1 - b1*c2)/determinant)
        y = round((a1*c2 - a2*c1)/determinant)
        return (x,y)
    else:
        return None

In [6]:
def get_angle(p1, p2, p3, p4, degrees=False):
    vec1 = (p2[0]-p1[0], p2[1]-p1[1])
    vec2 = (p4[0]-p3[0], p4[1]-p3[1])
    
    uvec1 = vec1 / np.linalg.norm(vec2)
    uvec2 = vec2 / np.linalg.norm(vec2)
    angle = np.arccos(np.clip(np.dot(uvec1, uvec2), -1.0, 1.0))
    
    if degrees:
        angle = angle*180/np.pi
        
    return angle

In [7]:
# Defining our color space
color_space = 'HSV'

# Reading threshhold values for the mask
threshold_vals = []
threshold_file = open(f'{color_space}_values', 'r')
for line in threshold_file.readlines():
    threshold_vals.append(np.array([int(x) for x in line.strip().split()]))
    
print(threshold_vals)

[array([54, 39,  0]), array([255, 255, 255])]


In [18]:
def get_binary_threshold(frame, threshold_values):
    # Masking the frame with the threshold values
    mask = cv2.inRange(frame, threshold_values[0], threshold_values[1])
    masked = cv2.bitwise_and(frame, frame, mask=mask)
    
    # Finding edges/contours
    #edges = cv2.Canny(masked,100,200)
    
    # Getting a thresolded image
    gray = masked[:,:,0]+masked[:,:,1]  #gray = H values + S values
    ret, gray = cv2.threshold(gray, 5, 255, cv2.THRESH_BINARY)
    
    return gray

In [19]:
def get_roi(frame, interest_percentage):
    x_end = frame.shape[1]
    y_end = round(frame.shape[0]*(1-interest_percentage))
    black_region = np.zeros((y_end, x_end))
    frame[0:y_end, 0:x_end] = black_region

In [20]:
def get_windows_values(frame, nonzerox, nonzeroy, x_values, n_windows, window_height, width_margin, min_recenter_pixels, downwards=False):    
    # list of points
    values = dict()
    
    # Start the next windows where the mean from the previous wnidow is
    follow_previous = False
    
    # Checking if x values are provided or should be found
    if len(x_values) == 1:
        values['x_points'] = np.zeros(n_windows, dtype=np.int16)
        values['x_points'][0] = x_values[0]
        follow_previous = True
        
    elif len(x_values) == n_windows:
        values['x_points'] = x_values
    else: raise ValueError('x_values must be either a single point o a list of values for each of the windows')
        
    values['y_points'] = np.zeros(n_windows, dtype=np.int16)
    values['corners'] = []
    values['windows'] = [False for _ in range(n_windows)]
    
    # Check if we're going upwards or downwards
    windows = range(n_windows)[::-1] if downwards else range(n_windows)
    
    for window in windows:        
        y_bottom = frame.shape[0] - (window+1)*window_height
        y_top = frame.shape[0] - window*window_height
        y_current = y_top-(y_top-y_bottom)//2
        x_left = values['x_points'][window] - width_margin
        x_right = values['x_points'][window] + width_margin
        
        # Getting the index of the non zero pixels in the window
        valid_idx = ((nonzeroy >= y_bottom) & (nonzeroy < y_top) & (nonzerox >= x_left) & (nonzerox < x_right)).nonzero()[0]
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(valid_idx) > min_recenter_pixels:
            values['x_points'][window] = int(np.mean(nonzerox[valid_idx]))
            values['windows'][window] = True # Saving the index of the window to use latter for fitting
            
        if follow_previous and window < n_windows-1: values['x_points'][window+1] = values['x_points'][window]
            
        # Saving the newly computed points
        values['y_points'][window] = y_current
        values['corners'].append(((x_left, y_bottom),(x_right, y_top)))
        
    return values

In [21]:
def get_bad_window_limit(windows):
    for i, window in enumerate(windows):
        if window: return i-1
    return len(windows)

def get_windows_with_histogram(frame, x_current, nwindows=10, min_recenter_pixels=25, width_margin=50):    
    # Window height
    window_height = frame.shape[0]//(2*nwindows)
    
    # Pixels that are non zero
    nonzero = frame.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    values = get_windows_values(frame=frame, nonzerox=nonzerox, nonzeroy=nonzeroy, x_values=[x_current], n_windows=nwindows, window_height=window_height, width_margin=width_margin, min_recenter_pixels=min_recenter_pixels)
    bad_limit = get_bad_window_limit(values['windows'])
    
    if  -1 < bad_limit < nwindows: # if there's at least 1 bad window and not all of them are bad
        good_nearest = values['x_points'][bad_limit+1] 
        fixed_values = get_windows_values(frame=frame, nonzerox=nonzerox, nonzeroy=nonzeroy, x_values=[good_nearest], n_windows=bad_limit+1, window_height=window_height, width_margin=width_margin, min_recenter_pixels=min_recenter_pixels, downwards=True)
        
        #Assigning our new fixed values 
        values['x_points'][:bad_limit+1] = fixed_values['x_points']
        values['corners'][:bad_limit+1] = fixed_values['corners']
        
        # Finally re-sweeping the windows through to adjust upper windows
        values = get_windows_values(frame=frame, nonzerox=nonzerox, nonzeroy=nonzeroy, x_values=[values['x_points'][0]], n_windows=nwindows, window_height=window_height, width_margin=width_margin, min_recenter_pixels=min_recenter_pixels)
    return values['x_points'], values['y_points'], values['corners'], values['windows']

In [22]:
def get_windows_with_polynomial(frame, fit, nwindows=10, min_recenter_pixels=25, width_margin=50):
    # Window height
    window_height = frame.shape[0]//(2*nwindows)
    
    # Getting our defined y values to compute the x ones
    y_vals = [int(frame.shape[0]-(window_height*i)-(window_height/2)) for i in range(nwindows)]
    
    x_vals = np.polyval(fit, y_vals).astype(int)
    
    # Pixels that are non zero
    nonzero = frame.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    values = get_windows_values(frame=frame, nonzerox=nonzerox, nonzeroy=nonzeroy, x_values=x_vals, n_windows=nwindows, window_height=window_height, width_margin=width_margin, min_recenter_pixels=min_recenter_pixels)
    return values['x_points'], values['y_points'], values['corners'], values['windows']

In [23]:
def fit_poly_points(x_points, y_points, degree=2, x_range=(0,100)):
    fit = np.polyfit(x_points, y_points, degree)
    
    x_vals = np.arange(x_range[0], x_range[1])
    
    try:
        y_vals = np.polyval(fit, x_vals)
    except TypeError as e:
        print(e)
        pass
    
    return fit, (x_vals, y_vals)

In [45]:
cap = cv2.VideoCapture(2)

# Defining the number of windows to use
nwindows = 15

# True if a section of lane was found per window
left_windows = [False for _ in range(nwindows)]
right_windows = [False for _ in range(nwindows)]

# Polynomial fit degree
degree = 3

while cap.isOpened():
    ret, frame =  cap.read()
    
    if frame is None:
        print('No frame')
        break
        
    # Cropping the image to remove logitech border
    frame = frame[75:frame.shape[0]-75, 15:frame.shape[1]-15, :]
    # Creating the frame were we're gonna see the regression and windows
    frame_debug = frame.copy()
        
    # Changing to different color spaces
    #lab_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Choosing the frame we are gonna use for the lane detection sliding windows -> gray; hough lines -> edges
    working_frame = get_binary_threshold(hsv_frame, threshold_vals)
    
    # Taking into account only certain percetage of the image (bottom half)
    get_roi(working_frame, 0.5)
    
    # Getting the window center points for both left and right lane
    # if there's already a fit for the lane, then use it to position the windows, otherwise find greateast probability with histogram 
    if sum(left_windows) < int(nwindows*0.3): 
        histogram = np.sum(working_frame[working_frame.shape[0]//2:, :working_frame.shape[1]//2], axis=0) # Half of the frame width
        left_xcurrent = np.argmax(histogram)

        try:
            (left_xpoints, y_points, left_corners, left_windows) = get_windows_with_histogram(working_frame, left_xcurrent, nwindows=nwindows, min_recenter_pixels=25, width_margin=50) 
        except Exeception as e:
            print(e)
    else:
        try:
            (left_xpoints, y_points, left_corners, left_windows) = get_windows_with_polynomial(working_frame, prev_left_fit, nwindows=nwindows, min_recenter_pixels=25, width_margin=50)
        except Exception as e:
            print(e)
            
    if sum(right_windows) < int(nwindows*0.3):
        histogram = np.sum(working_frame[working_frame.shape[0]//2:, working_frame.shape[1]//2:], axis=0) # Half of the frame width
        right_xcurrent = np.argmax(histogram) + working_frame.shape[1]//2

        try:
            (right_xpoints, y_points, right_corners, right_windows) = get_windows_with_histogram(working_frame, right_xcurrent, nwindows=nwindows, min_recenter_pixels=25, width_margin=50) 
        except Exeception as e:
            print(e)
    else:
        try:
            (right_xpoints, y_points, right_corners, right_windows) = get_windows_with_polynomial(working_frame, prev_right_fit, nwindows=nwindows, min_recenter_pixels=25, width_margin=50)
        except Exception as e:
            print(e)
    
    # Fitting a third degree polynomial curve to the points
    good_x_left = left_xpoints[left_windows]
    good_y_left = y_points[left_windows]
    good_x_right = right_xpoints[right_windows]
    good_y_right = y_points[right_windows]
    
    try:
        prev_left_fit, left_fit = fit_poly_points(good_y_left, good_x_left, degree=degree, x_range=(working_frame.shape[0]//2, working_frame.shape[0]))
        prev_right_fit, right_fit = fit_poly_points(good_y_right, good_x_right, degree=degree, x_range=(working_frame.shape[0]//2, working_frame.shape[0]))
        
        # Drawing polygon with left and right values
        poly_y = np.concatenate((left_fit[0], right_fit[0][::-1]), axis=None)
        poly_x = np.concatenate((left_fit[1], right_fit[1][::-1]), axis=None)

        # Format taken by the fillPoly function np.array([[x1,y1],[x2,y2],[x3,y3],...] )
        poly_points = np.array([[x, y] for x, y in zip(poly_x, poly_y)], dtype = np.int32)
        
        cv2.putText(img=frame_debug, text=f"left: {prev_left_fit}", org=(0,50), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(255,255,255), thickness=1, lineType=cv2.LINE_AA)
        cv2.putText(img=frame_debug, text=f"right: {prev_right_fit}", org=(0,100), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(255,255,255), thickness=1, lineType=cv2.LINE_AA)
        cv2.putText(img=frame_debug, text=f"merged: {prev_left_fit==prev_right_fit}", org=(0,150), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(255,255,255), thickness=1, lineType=cv2.LINE_AA)
        
    except Exception as e:
        print('Not enough points to fit')

    # Drawing windows and it's centers
    for i, val in enumerate(left_windows):
        color = (0,255,255) if val else (255,0,255)
        cv2.circle(frame_debug, (left_xpoints[i], y_points[i]), 2, color, 2)
        cv2.rectangle(frame_debug, (left_corners[i][0][0], left_corners[i][0][1]), (left_corners[i][1][0], left_corners[i][1][1]), (0,0,255), 2)
        
    for i, val in enumerate(right_windows):
        color = (0,255,255) if val else (255,0,255)
        cv2.circle(frame_debug, (right_xpoints[i], y_points[i]), 2 , (0,255,255), 2)
        cv2.rectangle(frame_debug, (right_corners[i][0][0], right_corners[i][0][1]), (right_corners[i][1][0], right_corners[i][1][1]), (0,0,255), 2)
    
    # Drawing the 3rd degree poly curve
    for i in range(len(left_fit[0])):
        #print((left_fit[1][i], left_fit[0][i]), (right_fit[1][i], right_fit[0][i]))
        cv2.circle(frame_debug, (int(left_fit[1][i]), int(left_fit[0][i])), 1, (255,255,255), 1)
        cv2.circle(frame_debug, (int(right_fit[1][i]), int(right_fit[0][i])), 1, (255,255,255), 1)
        
    # Drawing line in the middle of the screen
    cv2.line(frame_debug, (frame_debug.shape[1]//2, 0), (frame_debug.shape[1]//2, frame_debug.shape[0]), (255,255,255), 1)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q') or key == 27: break
    
    # Drawing it a new frame so we later merge them together with certain alpha
    lane_search_area = np.zeros(frame.shape, dtype=np.uint8)
    cv2.fillPoly(lane_search_area, [poly_points], (160,255,160))
    frame = cv2.addWeighted(src1=frame, alpha=0.6, src2=lane_search_area, beta=0.4, gamma = 0)
    
    cv2.imshow('frame', frame)
    cv2.imshow('debug', frame_debug)
    #cv2.imshow('edges', edges)
    cv2.imshow('gray', working_frame)
    
cap.release()
cv2.destroyAllWindows()

C:\Users\jprr2\AppData\Local\Temp/ipykernel_24024/1002746285.py:74: RankWarning: Polyfit may be poorly conditioned
  prev_right_fit, right_fit = fit_poly_points(good_y_right, good_x_right, degree=degree, x_range=(working_frame.shape[0]//2, working_frame.shape[0]))


Not enough points to fit
Not enough points to fit
Not enough points to fit
Not enough points to fit
Not enough points to fit
Not enough points to fit
Not enough points to fit


C:\Users\jprr2\AppData\Local\Temp/ipykernel_24024/1002746285.py:73: RankWarning: Polyfit may be poorly conditioned
  prev_left_fit, left_fit = fit_poly_points(good_y_left, good_x_left, degree=degree, x_range=(working_frame.shape[0]//2, working_frame.shape[0]))
C:\Users\jprr2\AppData\Local\Temp/ipykernel_24024/1002746285.py:74: RankWarning: Polyfit may be poorly conditioned
  prev_right_fit, right_fit = fit_poly_points(good_y_right, good_x_right, degree=degree, x_range=(working_frame.shape[0]//2, working_frame.shape[0]))
C:\Users\jprr2\AppData\Local\Temp/ipykernel_24024/1002746285.py:73: RankWarning: Polyfit may be poorly conditioned
  prev_left_fit, left_fit = fit_poly_points(good_y_left, good_x_left, degree=degree, x_range=(working_frame.shape[0]//2, working_frame.shape[0]))
C:\Users\jprr2\AppData\Local\Temp/ipykernel_24024/1002746285.py:73: RankWarning: Polyfit may be poorly conditioned
  prev_left_fit, left_fit = fit_poly_points(good_y_left, good_x_left, degree=degree, x_range=(work

Not enough points to fit
Not enough points to fit
Not enough points to fit
Not enough points to fit
Not enough points to fit


C:\Users\jprr2\AppData\Local\Temp/ipykernel_24024/1002746285.py:73: RankWarning: Polyfit may be poorly conditioned
  prev_left_fit, left_fit = fit_poly_points(good_y_left, good_x_left, degree=degree, x_range=(working_frame.shape[0]//2, working_frame.shape[0]))


...

Not enough points to fit
Not enough points to fit
Not enough points to fit
Not enough points to fit
Not enough points to fit
Not enough points to fit


C:\Users\jprr2\AppData\Local\Temp/ipykernel_24024/1002746285.py:73: RankWarning: Polyfit may be poorly conditioned
  prev_left_fit, left_fit = fit_poly_points(good_y_left, good_x_left, degree=degree, x_range=(working_frame.shape[0]//2, working_frame.shape[0]))
C:\Users\jprr2\AppData\Local\Temp/ipykernel_24024/1002746285.py:73: RankWarning: Polyfit may be poorly conditioned
  prev_left_fit, left_fit = fit_poly_points(good_y_left, good_x_left, degree=degree, x_range=(working_frame.shape[0]//2, working_frame.shape[0]))
C:\Users\jprr2\AppData\Local\Temp/ipykernel_24024/1002746285.py:73: RankWarning: Polyfit may be poorly conditioned
  prev_left_fit, left_fit = fit_poly_points(good_y_left, good_x_left, degree=degree, x_range=(working_frame.shape[0]//2, working_frame.shape[0]))
C:\Users\jprr2\AppData\Local\Temp/ipykernel_24024/1002746285.py:73: RankWarning: Polyfit may be poorly conditioned
  prev_left_fit, left_fit = fit_poly_points(good_y_left, good_x_left, degree=degree, x_range=(working_

Not enough points to fit
Not enough points to fit


C:\Users\jprr2\AppData\Local\Temp/ipykernel_24024/1002746285.py:74: RankWarning: Polyfit may be poorly conditioned
  prev_right_fit, right_fit = fit_poly_points(good_y_right, good_x_right, degree=degree, x_range=(working_frame.shape[0]//2, working_frame.shape[0]))


Not enough points to fit
Not enough points to fit
Not enough points to fit
Not enough points to fit
Not enough points to fit
Not enough points to fit
Not enough points to fit
Not enough points to fit


In [113]:
"""
    # ----------------------- HOUGH LINES METHOD ----------------------------#
    
    # Finding line segments
    lines = cv2.HoughLinesP(gray,1,np.pi/180,50,minLineLength=50,maxLineGap=10)
    if lines is not None:
        #print(len(lines))
        # Finding the longest line segment on both sides of the screen
        longest_left = [0, (0,0,0,0)] # Distance, (x1,y1,x2,y2)
        longest_right = [0, (0,0,0,0)] # Distance, (x1,y1,x2,y2)
        for line in lines:
            # Calculating the total line distance and saving the longest line
            x1,y1,x2,y2 = line[0]
            
            #cv2.line(frame, (x1,y1), (x2,y2), (255,255,0), 2)
            #cv2.putText(frame, f'{len(lines)}', org=(50,50), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(75,60,100), thickness=1, lineType=cv2.LINE_AA)
            
            dist = ((y2-y1)**2 + (x2-x1)**2)**0.5
            
            # Checking if line is in the left side
            if x1 <= edges.shape[1]//2 and x2 <= edges.shape[1]//2:
                if dist >= longest_left[0]:
                    longest_left[0] = dist
                    longest_left[1] = line[0]
            # Else in the right side
            else:
                if dist >= longest_right[0]:
                    longest_right[0] = dist
                    longest_right[1] = line[0]
                        
            
        # Drawing only the longest line on both sides
        #print(f'first: ({longest_left[1][0]},{longest_left[1][1]})', f'second: ({longest_left[1][2]},{longest_left[1][3]})')
        #cv2.line(frame, (longest_left[1][0],longest_left[1][1]), (longest_left[1][2],longest_left[1][3]), (0,255,0), 2)
        #cv2.line(frame, (longest_right[1][0],longest_right[1][1]), (longest_right[1][2], longest_right[1][3]), (0,255,0), 2)
        
    
        
        # ----------------------- HOUGH LINES METHOD ----------------------------#
        
        # - - - - - Technique No. 1 to find center reference - - - - - #
        # Finding and drawing the middle point on both lines (longest segments)
        middle_left = ((longest_left[1][0]+longest_left[1][2])//2, (longest_left[1][1]+longest_left[1][3])//2)
        middle_right = ((longest_right[1][0]+longest_right[1][2])//2, (longest_right[1][1]+longest_right[1][3])//2)
        cv2.circle(frame, middle_left, 5, (0,0,255), 5)
        cv2.circle(frame, middle_right, 5, (0,0,255), 5)
        # Lines between both points and middle point -> reference to follow
        center_reference = ((middle_left[0]+middle_right[0])//2, (middle_left[1]+middle_right[1])//2)
        cv2.line(frame, middle_left, middle_right, (255,0,0), 2)
        cv2.circle(frame, center_reference, 5, (255,255,255), 5)
        # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - #
        
        # - - - - - Technique No. 2 to find center reference - - - - - #
        # Lines between opposite sides of the longest segments
        # Checking which points are up or down
        if longest_left[1][1] > longest_left[1][3]:
            lower_left = (longest_left[1][0], longest_left[1][1])
            upper_left = (longest_left[1][2], longest_left[1][3])
        else: 
            lower_left = (longest_left[1][2], longest_left[1][3])
            upper_left = (longest_left[1][0], longest_left[1][1])
            
        if longest_right[1][1] > longest_right[1][3]:
            lower_right = (longest_right[1][0], longest_right[1][1])
            upper_right = (longest_right[1][2], longest_right[1][3])
        else: 
            lower_right = (longest_right[1][2], longest_right[1][3])
            upper_right = (longest_right[1][0], longest_right[1][1])
        
        # Drawing opposite lines
        cv2.line(frame, upper_left, lower_right, (0,255,255), 2)
        cv2.line(frame, lower_left, upper_right, (0,255,255), 2)
        #Drawing intersection point between both lines
        intersection_reference = find_intersection(upper_left, lower_right, lower_left, upper_right)
        if intersection_reference:
            cv2.circle(frame, intersection_reference, 5, (255,0,255), 5)
        # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - #
        
    else: pass #print('No lines detected')
    """ 

"\n    # ----------------------- HOUGH LINES METHOD ----------------------------#\n    \n    # Finding line segments\n    lines = cv2.HoughLinesP(gray,1,np.pi/180,50,minLineLength=50,maxLineGap=10)\n    if lines is not None:\n        #print(len(lines))\n        # Finding the longest line segment on both sides of the screen\n        longest_left = [0, (0,0,0,0)] # Distance, (x1,y1,x2,y2)\n        longest_right = [0, (0,0,0,0)] # Distance, (x1,y1,x2,y2)\n        for line in lines:\n            # Calculating the total line distance and saving the longest line\n            x1,y1,x2,y2 = line[0]\n            \n            #cv2.line(frame, (x1,y1), (x2,y2), (255,255,0), 2)\n            #cv2.putText(frame, f'{len(lines)}', org=(50,50), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(75,60,100), thickness=1, lineType=cv2.LINE_AA)\n            \n            dist = ((y2-y1)**2 + (x2-x1)**2)**0.5\n            \n            # Checking if line is in the left side\n            if x1 <= edge